# Sentencias Estructurales


Tablas:

[ ] 1.      Cantidad de sentencias por año

[ ] 2.      Cantidad de sentencias por tema y por año

[ ] 3.      Cantidad de sentencias por subtema y por año

[ ] 4.      Cantidad de sentencias por clase de asunto y por año

[ ] 5.      Cantidad de sentencias por magistrad@ que redacta

[ ] 6.      Recurrentes de sentencias por tema y por año

[ ] 7.      Recurridos de sentencias por tema y por año

[ ] 8.      Artículos citados en las sentencias por tema y por año

[ ] 9.      Leyes citadas en las sentencias por tema y por año

[ ] 10. Reglamentos citados en las sentencias por tema y por año

[ ] 11.  Instrumentos internacionales citados en las sentencias por tema y por año

[ ] 12.  Cantidad de sentencias que referencian otras sentencias por año

[ ] 13. Derechos fundamentales citados en las sentencias por tema y por año

[ ] 14. Resultado de las sentencias por año y por tema

[ ] 15. Cantidad de sentencias que determinan órdenes por año y por tema

[ ] 16.  Cantidad de sentencias que determinan plazos por año y por tema

[ ] 17.  Cantidad de sentencias que determinan dar seguimiento por año y por tema

[ ] 18.  Cantidad de sentencias que determinan condenas por año y por tema

[ ] 19.  Instituciones (personas y/o entes a los que la SC le ordena) por año y por tema

[ ] 20.  Sentencias por plazos definidos para su cumplimiento y por tema


In [1]:
%%capture --no-display
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
import os
os.chdir('/home/jovyan/Work/ej/paquetes/nlppen/notebooks/ReporteTecnico')

from reporte import *
from IPython.display import Latex


import sys
sys.path.insert(0, '../../')
from nlppen.descripcion import *
from pyspark.sql import functions as F



# Inicialización y Carga de datos
spark, sc = init_spark()
estructurales = cargar_datos(spark)

22/01/25 20:28:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
estructurales.seleccion.sdf.count()

507951

In [3]:
from pyspark.sql import functions as F



In [4]:
estructurales.seleccion.sdf #= pre_sdf
temas_sdf = spark.read.parquet('/home/jovyan/Work/ej/src/datasets/complete_prev').select('archivo','anno','tema', 'tema_prob', 'termino_ext', 'subtema', 'subtema_prob')
temas_df = estructurales.seleccion.sdf.withColumnRenamed('tema', 'tema_malo').withColumnRenamed('tema_prob', 'tema_prob_malo').join(temas_sdf.select('archivo','tema', 'tema_prob'), 'archivo')

In [5]:
pre_sdf = estructurales.seleccion.sdf
estructurales.seleccion.sdf = (pre_sdf
                               .withColumnRenamed('tema', 'tema_malo')
                               .withColumnRenamed('tema_prob', 'tema_prob_malo')
                               .withColumnRenamed('subtema', 'subtema_malo')
                               .withColumnRenamed('subtema_prob', 'subtema_prob_malo')
                               .join(temas_sdf.select('archivo','tema', 'tema_prob'), 'archivo')
                               .persist())

In [6]:
temas_sdf.count()

356687

In [15]:
(estructurales.seleccion.sdf
       .filter('termino_ext == "Con lugar" OR termino_ext == "Con lugar parcial"')
       .select(F.col('archivo').alias('Archivo'),
               F.col('id_sentencia').alias('id_sentencia'),
               F.col('num_doc_oficial').alias('num_doc_oficial'),
               F.col('fechahora_ext').alias('Fecha'),
               F.col('anno').alias('Año'),
               F.col('FechaSolicitud').alias('Fecha_solicitud'),
               F.col('expediente').alias('expediente'),
               F.col('redactor_ext').alias('Magistrado (a) redacta'),
               F.col('tipoResolucion_ext').alias('Clase_asunto'),
               F.col('voto_salvado').alias('Voto_salvado'),
               F.col('extension_sentencia').alias('Extension_sentencia'),
               F.col('extension_por_lo_tanto').alias('Extensión_por tanto'),
               F.col('tema').alias('Tema'),
#                F.col('subtema').alias('Subtema'),
#                F.col('FechaSolicitud').alias('Problema que busca solucionar'),
#                F.col('FechaSolicitud').alias('Grupo de personas afectado'),
#                F.col('FechaSolicitud').alias('Detalle "Se ordena"'),
               F.col('plazo').alias('Plazo (días)'),
#                F.col('seguimiento').alias('Detalle seguimiento'),
              )

       ).persist().limit(5).toPandas()

,Archivo,id_sentencia,num_doc_oficial,Fecha,Año,Fecha_solicitud,expediente,Magistrado (a) redacta,Clase_asunto,Voto_salvado,Extension_sentencia,Extensión_por tanto,Tema,Plazo (días)
0,14d28.html,sen-1-0007-85288,790,1990-07-12 09:05:00,1990,1978-05-28,24090,Mora Mora MIGRACION Obligación de otorgar stat...,Recurso de Amparo,False,5857,225,MIGRACION,0
1,1558b.html,sen-1-0007-87435,2546,1991-11-27 16:15:00,1991,NaT,1798B91,Baudrit- y,Recurso de Amparo,False,2983,543,PETICION,0
2,15a45.html,sen-1-0007-88645,1713,1992-06-24 14:39:00,1992,NaT,2152B91,Baudrit- y,Recurso de Amparo,False,4919,1026,TRABAJO,1
3,161f4.html,sen-1-0007-90612,500,1993-01-29 10:09:00,1993,NaT,3874E92,Sancho González,Recurso de Amparo,False,4282,632,TRABAJO,0
4,17094.html,None,None,1994-02-02 17:57:00,1994,1993-08-31,4845A93,Arguedas Ramírez,Recurso de Amparo,False,6468,627,PETICION,1


In [15]:
columnas_sencillas = ['archivo',
            'expediente',
            'num_resolucion',
            'id_sentencia',
            'num_doc_oficial',
             'anno',
             'fechahora_ext',
                         'FechaSolicitud',
             'redactor_ext',
             'tipoResolucion_ext',
             'termino_ext',
             'voto_salvado',
             'tema',
             'tema_prob',
             # Listas 1 elemento
             'Recurrente_ents',
             'Recurrido_ents',
             'extension_sentencia',
             'extension_por_lo_tanto',
             'seguimientoValue',
             'seguimiento',
             'se_ordena',
             'se_condena',
             'plan',
             'plazo',
]

columnas_listas = [
            # LISTAS 
#              'seguimientoExt',
             'Reglamento_ents',
             'Constitucion_ents',
             'Ley_ents',
             'EntidadPublica_ents',
             'plazosDefinidos',
             'se_ordena_Ent_Pub',
             'se_condena_Ent_Pub',
             'inst_internacionales',
             'derechos_Norm',
             'derechos_GenXPat',
             'derechos_Acotados',
             'derechos_General',
             'derechos_Fundamental',
             'derechos_Humano',
             'citasVotDate',
]
  
# Variables no incluídas    
#              'citasFechas',            
#              'expediente_oficial',            
#              'citasIDVoto',            
#              'Magistrado_ents',
#              'se_ordena_PER',
#              'se_ordena_LOC',
#              'se_ordena_ORG',
#              'se_ordena_MISC',
#              'se_ordena_GPE',
#              'se_condena_PER',
#              'se_condena_LOC',
#              'se_condena_ORG',
#              'se_condena_MISC',
#              'se_condena_GPE',
#              'termino_lst',
#              'CitaSentencias_ents',
#              'Redactor_ents',
#              'Fechahora_ents',
#              '_id',
#              'Cantón_ents',
#              'Fecha_ents',


sdf = (estructurales.seleccion.sdf
       .filter('termino_ext == "Con lugar" OR termino_ext == "Con lugar parcial"')
       .withColumn('Recurrente_ents', F.col("Recurrente_ents").getItem(0))
       .withColumn('Recurrido_ents', F.col("Recurrido_ents").getItem(0))
       .withColumnRenamed('Constitución_ents', 'Constitucion_ents')
       ).persist()

In [17]:
sdf.select(columnas_sencillas).coalesce(1).write.mode('overwrite').csv('SentEstruct_individuales.csv', header=True, sep=';')

In [5]:
from pyspark.sql import Row
from collections import Counter
from datetime import datetime
from tqdm.auto import tqdm


for columna in tqdm(columnas_listas):
    elems_selecc = (sdf
                    .select(columna)
                    .filter(f'{columna} is not Null')
                    .filter('termino_ext == "Con lugar" OR termino_ext == "Con lugar parcial"')
                    .select(F.explode(columna).alias(columna))
                    .groupby(columna)
                    .count()
                    .sort(F.desc('count'))
                    .filter('count > 5')
                    ).toPandas()

    elems_selecc['count'] = 0
    elems_selecc = elems_selecc.set_index(columna).to_dict('dict')['count']

    (sdf
     .select(*columnas_sencillas, columna)
     .filter(f'{columna} is not Null')
     .filter('termino_ext == "Con lugar" OR termino_ext == "Con lugar parcial"')
     .select('archivo', F.explode(columna).alias(columna))
     .filter(F.col(columna).isin(list(elems_selecc.keys())))
     ).toPandas().to_excel(f'SentEstructurales_{columna}_{datetime.now().strftime("%Y-%m-%d")}.xlsx')

  0%|          | 0/15 [00:00<?, ?it/s]

AnalysisException: The number of aliases supplied in the AS clause does not match the number of columns output by the UDTF expected 2 aliases but got citasVotDate 

In [ ]:
    
    def func(row):
        result = elems_selecc.copy()
        for elem in row[columna]:
            if elem in result:
                result[elem] += 1 
        return Row(archivo=row.archivo, **result)

    df = (sdf
     .select('archivo',columna)
     .filter(f'{columna} is not Null')
     .filter('termino_ext == "Con lugar" OR termino_ext == "Con lugar parcial"')
     .rdd
     .map(func)
    ).toDF().coalesce(1).write.mode('overwrite').csv(columna,header = 'true')#.toPandas()#.to_excel('Prueba2.xlsx')

In [ ]:
# Lista General de sentencias para el excel
columnas = ['archivo',
            'expediente', 'anno', 'fechahora_ext', 'redactor_ext', 'tipoResolucion_ext',
            'termino_ext', 'voto_salvado', 'tema', 'tema_prob', 'Recurrente_ents',
            'Cantón_ents', 'Fecha_ents', 'Constitución_ents', 'Redactor_ents','Ley_ents',
            'EntidadPublica_ents', 'Fechahora_ents', 'CitaSentencias_ents',
            'Recurrido_ents', 'Reglamento_ents', 'Magistrado_ents', 'seguimiento',
            'se_ordena', 'plan', 'plazo', 'se_ordena_PER', 'se_ordena_LOC', 
            'se_ordena_ORG', 'se_ordena_MISC', 'se_ordena_GPE', 'se_ordena_Ent_Pub',
            'extension_sentencia', 'extension_por_lo_tanto', 'plazosDefinidos',
            'FechaSolicitud', 'plazosDefinidos', 'dias_tramite', 'num_resolucion'
            ]

preprocesar_plazos(estructurales)
descr = Descripcion(estructurales.seleccion, './extraccion')
descr.guardar_lista(columnas, sheet='Lista de Sentencias con lugar')

In [ ]:
columnas = ['archivo',
            'expediente', 'anno', 'fechahora_ext', 'seguimiento',
            'se_ordena', 'plan', 'plazo', 'se_ordena_PER', 'se_ordena_LOC', 
            'se_ordena_ORG', 'se_ordena_MISC', 'se_ordena_GPE', 'se_ordena_Ent_Pub',
            'extension_sentencia', 'extension_por_lo_tanto', 'plazosDefinidos',
            'FechaSolicitud', 'plazosDefinidos', 'dias_tramite', 'num_resolucion'
            ]


# estructurales.seleccion.sdf.where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').select(*columnas).toPandas().to_excel('datos02.xlsx')

## Introducción

En este reporte se exploran resultados preliminares de la aplicación de los criterios de clasificación de sentencias estructurales en el corpus de resoluciones de la Sala Constitucional de Costa Rica. Estos criterios están definidos en el documento de trabajo _"Propuesta de definición y operacionalización de sentencias estructurales"_ (Estado de la Justicia, 2021), en donde los criterios se agrupan las tres dimensiones: contenido, forma y calidad.

### Dimensión Contenido
 
 
| Num. | Característica                                                                | Operacionalización               | Unidad de registro                               |
| ---- |:------------------------------------------------------------------------------|:----------------------------------|:--------------------------------------------------|
| CO01  | Busca solucionar un problema estructural, reiterado y serio                   | Problema estructural que atiende | Listado de problemas estructurales a identificar |
| CO02  | Se centra en la protección de temáticas relacionados con los derechos humanos | 1. Se menciona un derecho humano afectado? <br> 2. ¿Cuál? | 1. Sí y no.  <br> 2. Listado de derechos humanos a identificar |
| CO03  | Afecta a un grupo de personas, más allá de quién (es) presentaron el recurso | 1. En la sección general hay más de un demandante (un nombre). <br> 2. Por lo tanto menciona un grupo de la población específico | 1. Sí o no.  | Contien la palabra indígenas, personas con discapacidad, privados de libertad, vecinos, estudiantes, pacientes, etc. |
| CO04  | Sirve como referente en la resolución de un elevado número de casos similares o idénticos |  1. Es citado en otras resoluciones. <br> 2. ¿Cuáles? | 1.Número de veces que se cita en otras resoluciones. <br> 2.Expedientes en los cuales se cita. |
| CO05  | Establece como una condición dar seguimiento para verificar el cumplimiento efectivo de dicha sentencia | Contiene la palabra "Seguimiento" | 1. Sí o no. <br> 2. (Informes??, planificación??) |
 
 

### Dimensión Forma

| Num. | Característica  | Operacionalización | Unidad de registro   |
| ---- |:----------------|--------------------|----------------------|
| FO01 | Implica órdenes/disposiciones de cumplimiento obligatorio  | Contiene un "Se ordena" <br> (?? Con un plazo)  | Sí o no.   |
| FO02 | Señala qué debe hacerse y a quién (persona o ente) le corresponde realizar dicha acción  | Después de la palabra “se ordena” se menciona un nombre de la persona o institución  | Extraer el nombre completo.   |
| FO03 | Define plazos para el cumplimiento de las órdenes/disposiciones emitidas  | En el Por Tanto se menciona la palabra “meses”  | Extraer las palabras o números antes de meses.   |
| FO04 | Se registran declaraciones/pronunciamientos repetidos por parte de la Sala Constitucional  | Cantidad de referencia a otras sentencias similares o iguales  | Número de referencias <br> Número de expediente de las sentencias a las cuales se hace referencia  |
| FO05 | Señala que para el cumplimiento del problema estructural se debe elaborar un plan  | Aparece la palabra “plan”  | Sí o no  |
| FO06 | Hay más de una institución involucrada en la ejecución y supervisión del cumplimiento de la sentencias  | ¿Cuántas instituciones están involucradas en la ejecución y supervisión del cumplimiento de la sentencias?  | Sí o no <br> Número de instituciones  |

### Dimensión Calidad
| Característica   | Operacionalización | Unidad de registro   |
|:-----------------|--------------------|----------------------|
| Extensión de la sentencia  | Cantidad de palabras (entre X y Y)  | Números absolutos  |
| Tiempo que tomó emitir la resolución  | Meses que tomó emitir la resolución (entre X y Y)  | Fecha de presentación del recurso y fecha del voto  |
| Uso de lenguaje jurídico y técnico  | Cantidad de términos jurídicos y técnicos únicos citados  | Términos únicos en el texto.  |
| Profundidad de las argumentaciones expuestas  |    |   |
| Cantidad de fuentes/autores citados  | En el Considerando: se mencionan autores, líneas filosóficas o literatura   | Incluye las palabras doctrina, literatura,  |
| Jurisprudencia citada  | En el Considerando se citan instrumentos internacionales referidos, u organismos internacionales, incluyendo cortes internacionales, otros tribunales constitucionales  | Cantidad de instrumentos. <br> <br>Listado de instrumentos citados. <br> <br>Listado de organizaciones citadas  |
| Redactada por los magistrados y magistradas propietarios  | Redacta un propietario. <br> <br> <br>Votación la realizan  magistrados (as) propietarios (as)   | Si o no.  <br> <br>Cantidad de magistrados (as) propietarios (as) que votan  |

# Dimensión Contenido

## Criterio CO5. Seguimiento

In [ ]:
seguimiento_total = estructurales.seleccion.sdf.where('seguimiento != 0').count()

Latex(f'''
El objetivo es identificar si en el portanto de la sentencia se hace referencia
a alguna estrategia de seguimiento, ya sea por la misma Sala o por un tercero.


Para obtener la información, se realiza una búsqueda de la palabra seguimiento
en el portanto pero no se obtienen resultados concluyentes: solo se obtienen
{seguimiento_total} sentencias con la palabra, pero la mayor parte no hacen
referencia a un plan de seguimiento en particular.

''')

# Dimensión Forma

## FO01. Se ordena

In [ ]:
se_ordena_total = (estructurales
                    .seleccion.sdf
                    .where('se_ordena != 0')
                    .count())


con_lugar_total = (estructurales
                     .seleccion.sdf
                     .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
                     .count())

porcentaje_seordena = 100*se_ordena_total/con_lugar_total

In [ ]:
porcentaje_seordena

En las sentencias "con lugar", normalmente se le ordena a los recurridos alguna acción relacionada a la naturaleza de la sentencia. Esta orden se da a personas particulares, en puestos públicos o pertenecientes a alguna institución o entidad. 

In [ ]:
Latex(f'''
Del total de sentencias "con lugar" o "con lugar parcial", el {porcentaje_seordena:1.2f}%, manteniendo un comportamiento homogéneo durante todos los años (Figura 1).
''')

In [ ]:
grafico_se_ordena_anno(estructurales)

## FO02. Se señala qué debe hacerse y a quién

Instituciones que aparecen en se ordena

In [ ]:
df_se_ordena_LOC = (estructurales.seleccion.sdf
 .where(estructurales.seleccion.sdf.se_ordena_LOC.isNotNull())
 .select(F.explode('se_ordena_LOC').alias('se_ordena_LOC'), 'anno')
 .groupby('se_ordena_LOC')
 .count()
 .sort('count')
).toPandas()
descr.guardar_df(df_se_ordena_LOC, sheet='Se Ordena LOC (Sin limpiar)')
df_se_ordena_LOC[:50]


Plazo en el cumplimiento de la orden

In [ ]:
df_se_ordena_Ent_Pub = (estructurales.seleccion.sdf
 .where(estructurales.seleccion.sdf.se_ordena_Ent_Pub.isNotNull())
 .select(F.explode('se_ordena_Ent_Pub').alias('se_ordena_Ent_Pub'), 'anno')
 .groupby('se_ordena_Ent_Pub')
 .count()
 .sort('count')
).toPandas()

descr.guardar_df(df_se_ordena_Ent_Pub, sheet='Se Ordena Entidades Públicas')
df_se_ordena_Ent_Pub.columns = ['Instituciones', 'Cantidad de menciones']
df_se_ordena_Ent_Pub[:20].set_index('Instituciones').plot.barh(figsize=(14,10), title="Instituciones mencionadas junto a 'Se ordena'")
df_se_ordena_Ent_Pub.to_excel('EntidadesPublicas.xlsx')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

fig = plt.figure( figsize=(14,10))
fig.suptitle('Histograma de la extención de las sentencias')

ax1 = fig.add_subplot(221)
(estructurales.seleccion.sdf.select('extension_sentencia')
).toPandas().plot.hist(bins=1000, xlim=(0,100000), ax=ax1, title="Sentencias sin filtrar", ylabel='Frecuencia', xlabel='Cantidad de palabras' )

ax2 = fig.add_subplot(222)
(estructurales
 .seleccion.sdf
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .select('extension_sentencia')
).toPandas().plot.hist(bins=1000, xlim=(0,100000), ax=ax2, title="Solo sentencas 'con lugar' y 'con lugar parcial'" , ylabel='Frecuencia', xlabel='Cantidad de palabras')


ax3 = fig.add_subplot(223)
(estructurales
 .seleccion.sdf
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .where('se_ordena == 1')
 .select('extension_sentencia')
).toPandas().plot.hist(bins=1000, xlim=(0,100000), ax=ax3,  title="Sentencias con órdenes en el portanto" , ylabel='Frecuencia', xlabel='Cantidad de palabras')

fig.show()

In [ ]:
ax = (estructurales
 .seleccion.sdf
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .select('extension_sentencia')
 .where('extension_sentencia < 100000')
).toPandas().plot.hist(bins=70, 
                       figsize=(12,6),
                       xlim=(0,60000),
                      legend=False)
ax.set_xlabel('Cantidad de palabras')
ax.set_ylabel('Frecuencia')
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x)).replace(',','.')))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x)).replace(',','.')))

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker

sns.set(font='Franklin Gothic Book', rc={
 'axes.axisbelow': False,
 'axes.grid': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'dimgrey',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
sns.set_context("notebook", rc={"font.size":16,
                                "axes.titlesize":16,
                                "axes.labelsize":10})


# fig = plt.figure( figsize=(14,10))
# fig.suptitle('Histograma de la extención de los portanto')

# ax1 = fig.add_subplot(221)
# (estructurales
#  .seleccion.sdf
#  .select('extension_por_lo_tanto')
#  .where('extension_por_lo_tanto < 10000')
# ).toPandas().plot.hist(bins=1000, xlim=(0,6000), ax=ax1, title="Sentencias sin filtrar", ylabel='Frecuencia', xlabel='Cantidad de palabras')


ax = (estructurales
 .seleccion.sdf
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .select('extension_por_lo_tanto')
 .where('extension_por_lo_tanto < 10000')
).toPandas().plot.hist(bins=70, 
                       figsize=(12,6),
                       xlim=(0,4000),
                      legend=False)
ax.set_xlabel('Cantidad de palabras')
ax.set_ylabel('Frecuencia')
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x)).replace(',','.')))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x)).replace(',','.')))



# ax3 = fig.add_subplot(223)
# (estructurales
#  .seleccion.sdf
#  .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
#  .where('se_ordena == 1')
#  .where('extension_por_lo_tanto < 10000')
#  .select('extension_por_lo_tanto')
# ).toPandas().plot.hist(bins=1000, xlim=(0,6000), ax=ax3,  title="Sentencias con órdenes en el portanto" , ylabel='Frecuencia', xlabel='Cantidad de palabras')

In [ ]:
cant_con_plazo = (estructurales
 .seleccion.sdf             
 .where('plazosDefinidos is not Null')
 .where('se_ordena == 1')
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
).toPandas()

cant_con_lugar = (estructurales
 .seleccion.sdf             
 .where('se_ordena == 1')
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
).count()


cant_con_plazo.shape[0]/cant_con_lugar


In [ ]:
plazo = (estructurales
 .seleccion.sdf             
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .select('anno','plazosDefinidos')
).toPandas()

datos = (plazo.reset_index()
 .groupby('anno')
 .agg({'index':'count', 'plazosDefinidos': 'count'})
 .rename(columns={'index':'Cantidad con lugar y con lugar parcial', 'plazosDefinidos': 'Sentencias con plazos'}))
ax = (datos
 .plot.bar(figsize=(16,8), xlabel='Años', ylabel='Cantidad', width=0.8))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x)).replace(',','.')))

(plazo.reset_index()
 .groupby('anno')
 .agg({'index':'count', 'plazosDefinidos': 'count'})
 .rename(columns={'index':'Cantidad con lugar y con lugar parcial', 'plazosDefinidos': 'Sentencias con plazos'})
 .to_excel('Datos_plazos.xlsx'))

datos

In [ ]:
(estructurales
 .seleccion.sdf             
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .where('se_ordena != 0 ')
 .where('plazosDefinidos is not NULL ')
 .select('anno', 'se_ordena', 'plazosDefinidos')
 .groupby('anno').count()
 .sort('anno')
).toPandas()

In [ ]:

t_tramite = estructurales.seleccion.sdf.where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').select('dias_tramite').dropna().toPandas()

In [ ]:
import seaborn as sns
import numpy as np

sns.set(font='Franklin Gothic Book', rc={
 'axes.axisbelow': False,
 'axes.grid': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'dimgrey',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
sns.set_context("notebook", rc={"font.size":16,
                                "axes.titlesize":20,
                                "axes.labelsize":10})

def show_values_on_bars(axs, h_v="v", space=0.4):
    def _show_on_single_plot(ax):
        if h_v == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height()
                value = int(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif h_v == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height()/2 - 0.05
                value = int(p.get_width())
                ax.text(_x, _y, '{:,}'.format(int(value)), ha="left", size=12 )

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)
        

import matplotlib.ticker as ticker


datos = pd.DataFrame([{
    'Más de 4 años': t_tramite.query(f'dias_tramite > {365*4}  ').shape[0],
    'De 2 años a 4 años': t_tramite.query(f'dias_tramite > {365*2} and dias_tramite <= {365*4} ').shape[0],
    'De 1 año a 2 años': t_tramite.query(f'dias_tramite > {365} and dias_tramite <= {365*2} ').shape[0],
    'De 120 días a 1 año': t_tramite.query('dias_tramite > 120 and dias_tramite <= 365 ').shape[0],
    'De 60 días a 120 días': t_tramite.query('dias_tramite > 60 and dias_tramite <= 120').shape[0],
    'Menos de 60 días': t_tramite.query('dias_tramite > 30 and dias_tramite <= 60').shape[0],
    'Menos de 30 días': t_tramite.query('dias_tramite <= 30').shape[0],
}]).T
ax = datos.plot.barh(legend=False, figsize=(12,8))

show_values_on_bars(ax, h_v="h", space=4)

ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,}'.format(int(x))))
ax.set_xlabel('Cantidad de Sentencias')
t_tramite.shape[0]
datos

In [ ]:
descr.export.guardar('estructurales_crudo[28-10-21]-corret')

In [ ]:
redactores = (estructurales.seleccion.sdf
 .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
 .select('redactor_ext')
 .groupby('redactor_ext')
 .count()
#  .withColumn('Magistrado', F.explode('Redactor_ents'))
).toPandas()

redactores.to_excel('lista_redactores.xlsx')

# Red de Citas

In [ ]:
s = estructurales.seleccion.sdf

edgelist = (s
 .select( 'id_sentencia', 'tema', 'tema_prob', 'fechahora_ext', F.year('fechahora_ext').alias('anno'), F.explode_outer(s.citasIDVoto).alias("Target", "cita_num_voto"))
 .filter(s.id_sentencia.isNotNull())
 .filter(s.fechahora_ext.isNotNull())
 .filter(s.citasIDVoto.isNotNull())
 .withColumnRenamed('id_sentencia', 'Source')
).toPandas()#.to_csv('RED_COMPLETA.csv', index=False)

edgelist.count()

In [ ]:
# edgelist = (estructurales.seleccion.sdf
#             .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
#             .select('CitaSentencias_ents', 'num_resolucion', 'anno', 'fechahora_ext')
#  .withColumn('citador', F.explode('CitaSentencias_ents'))
#  .withColumn('citador', F.regexp_replace('citador', "[^\d]", ""))
#  .withColumnRenamed('num_resolucion', 'citada')
#  .select('citador', 'citada', 'anno', 'fechahora_ext')
# ).toPandas()

In [ ]:
edgelist = (estructurales.seleccion.sdf
            .where('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
            .select('CitaSentencias_ents', 'num_resolucion', 'anno', 'fechahora_ext')
 .withColumn('citada', F.explode('CitaSentencias_ents'))
 .withColumn('citada', F.regexp_replace('citada', "[^\d]", ""))
 .withColumnRenamed('num_resolucion', 'citador')
 .select('citador', 'citada', 'anno', 'fechahora_ext')
).toPandas()



In [ ]:
edgelist.citada.value_counts().to_frame().to_excel('Lista_sentencias_citadas.xlsx')

In [ ]:
import networkx as nx

G = nx.DiGraph()
for i, elrow in edgelist.iterrows():
    G.add_edge(elrow['Source'], elrow['Target'])

edgelist['fechahora_ext'] = edgelist['fechahora_ext'].astype(str)
edgelist['order'] = edgelist.anno - 1989 
att = edgelist[['Source','tema', 'tema_prob', 'fechahora_ext', 'anno', 'order']].drop_duplicates().set_index('Source').T.to_dict()
nx.set_node_attributes(G, att)

In [ ]:
G.number_of_nodes()

In [ ]:
nx.write_gexf(G, "RED_COMPLETA_SENTENCIAS.gexf")